In [ ]:
pip install pandas transformers torch sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm

# Load the Helsinki model for Marathi → English
model_name = "Helsinki-NLP/opus-mt-mr-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.eval()

# Read CSV
df = pd.read_csv("cleaned_sample.csv")

# Check if a string contains Devanagari (Marathi)
def is_marathi(text):
    return isinstance(text, str) and any('\u0900' <= ch <= '\u097F' for ch in text)

# Translate function
def translate_mr(text):
    if not is_marathi(text):
        return text
    try:
        inputs = tokenizer(text, return_tensors="pt", padding=True)
        with torch.no_grad():
            tokens = model.generate(**inputs, max_length=256)
        return tokenizer.decode(tokens[0], skip_special_tokens=True)
    except:
        return text

# Translate column names
new_columns = [translate_mr(col) for col in df.columns]
df.columns = new_columns

# Translate all cells
for col in tqdm(df.columns, desc="Translating columns"):
    df[col] = df[col].apply(translate_mr)

# Save the output
df.to_csv("translated_sample.csv", index=False)
print("✅ Done! Saved to 'translated_sample.csv'")


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Translating columns: 100%|██████████| 39/39 [21:16<00:00, 32.74s/it]

✅ Done! Saved to 'translated_sample.csv'
